<a href="https://colab.research.google.com/github/kattens/Protein-Interaction-with-LLMs/blob/main/second_part_model_after_the_protbert_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import torch

In [ ]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CUDA not available")

# Set up the appropriate device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')


False
CUDA not available
Using device: cpu


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pairs_df = pd.read_csv('/content/drive/MyDrive/Pairs_df.csv')

In [ ]:
# Truncate each specified column to a maximum length of 500 characters
columns = ['masked_sequence_A', 'masked_sequence_B', 'Sequence_A', 'Sequence_B']
for col in columns:
    pairs_df[col] = pairs_df[col].apply(lambda x: x[:500] if len(x) > 500 else x)

# Find the longest string by length
pairs_df['Length'] = pairs_df['masked_sequence_A'].apply(len)
longest_string = pairs_df.loc[pairs_df['Length'].idxmax(), 'masked_sequence_A']
print(len(longest_string))


500


In [ ]:
pairs_df

,pair_id,File Name A,File Name B,masked_sequence_A,masked_sequence_B,coords_A,coords_B,Embeddings_A,Sequence_A,Embeddings_B,Sequence_B,tokenized_sequence_A,tokenized_sequence_B,tokenized_masked_sequence_A,tokenized_masked_sequence_B,sum_tokenized_sequence_A,sum_tokenized_sequence_B,Length
0,1H0J,1H0J_A,1H0J_B,---NKLVP-FYK-CPAGKNLCYKMFMVATPKVPVKRGCIDVCPKSS...,---NKLVPLF-----------YKMFMVATPKVPVKRGCIDVCPKSS...,"[(0.6666441154110079, 0.2338360596696657, -0.3...","[(-0.35362122723998757, 0.09592292335495496, -...","tensor([ 0.3624, -0.8023, -1.2588, 1.9488, -0...",LKCNKLVPLFYKTCPAGKNLCYKMFMVATPKVPVKRGCIDVCPKSS...,"tensor([ 0.8819, -0.2896, -0.0102, 0.5890, 0...",LKCNKLVPLFYKTCPAGKNLCYKMFMVATPKVPVKRGCIDVCPKSS...,"[10, 9, 2, 12, 9, 10, 18, 13, 10, 5, 20, 9, 17...","[10, 9, 2, 12, 9, 10, 18, 13, 10, 5, 20, 9, 17...","[0, 0, 0, 12, 9, 10, 18, 13, 0, 5, 20, 9, 0, 2...","[0, 0, 0, 12, 9, 10, 18, 13, 10, 5, 0, 0, 0, 0...","[10, 9, 2, 24, 18, 20, 36, 26, 10, 10, 40, 18,...","[10, 9, 2, 24, 18, 20, 36, 26, 20, 10, 20, 9, ...",60
1,1H0J,1H0J_A,1H0J_C,----KL-----KTCPAGKNLCYKMFMVATP--PVKRGCIDVCPKSS...,--CNKLVPLFYK----------KMFMVATPKVPVKRG-------SS...,"[(0.6666441154110079, 0.2338360596696657, -0.3...","[(0.16223801216970127, -0.4052887381516026, 0....","tensor([ 0.3624, -0.8023, -1.2588, 1.9488, -0...",LKCNKLVPLFYKTCPAGKNLCYKMFMVATPKVPVKRGCIDVCPKSS...,"tensor([-0.5326, 1.6610, -1.1897, -0.3621, 0...",LKCNKLVPLFYKTCPAGKNLCYKMFMVATPKVPVKRGCIDVCPKSS...,"[10, 9, 2, 12, 9, 10, 18, 13, 10, 5, 20, 9, 17...","[10, 9, 2, 12, 9, 10, 18, 13, 10, 5, 20, 9, 17...","[0, 0, 0, 0, 9, 10, 0, 0, 0, 0, 0, 9, 17, 2, 1...","[0, 0, 2, 12, 9, 10, 18, 13, 10, 5, 20, 9, 0, ...","[10, 9, 2, 12, 18, 20, 18, 13, 10, 5, 20, 18, ...","[10, 9, 4, 24, 18, 20, 36, 26, 20, 10, 40, 18,...",60
2,1H0J,1H0J_B,1H0J_C,----KLV---------------KMFMVATPKVPVKRG----CPKSS...,----KLVP--------------KMFMVATPKVPVKR------PKSS...,"[(-0.35362122723998757, 0.09592292335495496, -...","[(0.16223801216970127, -0.4052887381516026, 0....","tensor([ 0.8819, -0.2896, -0.0102, 0.5890, 0...",LKCNKLVPLFYKTCPAGKNLCYKMFMVATPKVPVKRGCIDVCPKSS...,"tensor([-0.5326, 1.6610, -1.1897, -0.3621, 0...",LKCNKLVPLFYKTCPAGKNLCYKMFMVATPKVPVKRGCIDVCPKSS...,"[10, 9, 2, 12, 9, 10, 18, 13, 10, 5, 20, 9, 17...","[10, 9, 2, 12, 9, 10, 18, 13, 10, 5, 20, 9, 17...","[0, 0, 0, 0, 9, 10, 18, 0, 0, 0, 0, 0, 0, 0, 0...","[0, 0, 0, 0, 9, 10, 18, 13, 0, 0, 0, 0, 0, 0, ...","[10, 9, 2, 12, 18, 20, 36, 13, 10, 5, 20, 9, 1...","[10, 9, 2, 12, 18, 20, 36, 26, 10, 5, 20, 9, 1...",60
3,1H1L,1H1L_B,1H1L_D,SQTIDKINSCYPLFEQD-YQELFR-KR------DAQ-VQ--FA---...,SQTIDKINSCYPLFEQDEYQ-LFR-KR----AHDAQRVQEVFA---...,"[(1.0335813771762299, -0.07467590563123697, 0....","[(1.596220592379937, -1.6520011412450177, 0.58...","tensor([ 0.2023, 0.2512, -2.8480, 0.2294, -0...",SQTIDKINSCYPLFEQDEYQELFRNKRQLEEAHDAQRVQEVFAWTT...,"tensor([ 1.1608e+00, 1.7323e+00, -1.1385e+00,...",SQTIDKINSCYPLFEQDEYQELFRNKRQLEEAHDAQRVQEVFAWTT...,"[16, 14, 17, 8, 3, 9, 8, 12, 16, 2, 20, 13, 10...","[16, 14, 17, 8, 3, 9, 8, 12, 16, 2, 20, 13, 10...","[16, 14, 17, 8, 3, 9, 8, 12, 16, 2, 20, 13, 10...","[16, 14, 17, 8, 3, 9, 8, 12, 16, 2, 20, 13, 10...","[32, 28, 34, 16, 6, 18, 16, 24, 32, 4, 40, 26,...","[32, 28, 34, 16, 6, 18, 16, 24, 32, 4, 40, 26,...",500
4,1H1Y,1H1Y_A,1H1Y_B,------SMLSSDFANLAAEADRM-R-------MDIMDGHFVPNLTI...,-----PSMLSSDFANLAAEADRM---------MDIMDGHFVPNLTI...,"[(-0.9165413095431918, -0.872759328495862, 0.2...","[(-0.5081293394188968, -0.7043022970351518, -1...","tensor([ 1.1525e-01, 1.9326e+00, -1.5379e-01,...",AAKIAPSMLSSDFANLAAEADRMVRLGADWLHMDIMDGHFVPNLTI...,"tensor([-5.6799e-01, 9.8331e-01, 3.2209e-01,...",AAKIAPSMLSSDFANLAAEADRMVRLGADWLHMDIMDGHFVPNLTI...,"[1, 1, 9, 8, 1, 13, 16, 11, 10, 16, 16, 3, 5, ...","[1, 1, 9, 8, 1, 13, 16, 11, 10, 16, 16, 3, 5, ...","[0, 0, 0, 0, 0, 0, 16, 11, 10, 16, 16, 3, 5, 1...","[0, 0, 0, 0, 0, 13, 16, 11, 10, 16, 16, 3, 5, ...","[1, 1, 9, 8, 1, 13, 32, 22, 20, 32, 32, 6, 10,...","[1, 1, 9, 8, 1, 26, 32, 22, 20, 32, 32, 6, 10,...",219
...,

Create dataset class that handles both global sequences and local sequences for protein pairs, and potentially prepares for the inclusion of 3D structural data

#the base Model (without coordinates at this point):

  ### Modeling Interactions:
  The mdel could be trained to recognize which amino acids interact by learning representations of local sequences that highlight these interactions. During training, the MLM objective helps the model learn contextual embeddings that are rich in information about which amino acids tend to be near each other and under what structural contexts they interact.

  ### Attention Mechanism:
   The custom attention mechanism can be used to weigh the importance of different amino acids in the global context when predicting the masked amino acids in the local sequences. This allows your model to focus more on the parts of the global sequences that are relevant to the interactions highlighted by the local sequences.

  ### Utilizing Global Sequences:
  While the local sequences are your primary interest, the global sequences provide the context necessary for your model to understand the broader environment in which the interactions occur. Even during prediction, you should feed the model the global sequences to utilize the learned context.

  #### This modular design not only meets your current requirements but also provides a scalable framework to incorporate additional dimensions of protein sequence data analysis in the future


In [ ]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
"""
try:
    # Initialize the ProtBERT tokenizer
    tokenizer = AutoTokenizer.from_pretrained('Rostlab/prot_bert')

    # Define special tokens for entities
    special_tokens = ['[ENTITY1]', '[ENTITY2]']

    # Add special tokens to the tokenizer
    tokenizer.add_special_tokens({'additional_special_tokens': special_tokens})

    # Check if the special tokens were added successfully
    print(f"Token '[ENTITY1]' has ID: {tokenizer.convert_tokens_to_ids('[ENTITY1]')}")
    print(f"Token '[ENTITY2]' has ID: {tokenizer.convert_tokens_to_ids('[ENTITY2]')}")

    # Initialize the ProtBERT model configured for Masked Language Modeling
    model = AutoModelForMaskedLM.from_pretrained('Rostlab/prot_bert')
    model.resize_token_embeddings(len(tokenizer))  # Adjust the model's embedding size to accommodate new tokens
    print('Token embeddings resized to accommodate new tokens.')

except Exception as e:
    print(f"An error occurred when initializing the tokenizer or model: {e}")

# Helper function to convert numerical token IDs back to their textual representation
def ids_to_text(ids):
    return ' '.join(tokenizer.convert_ids_to_tokens(ids))

# Check the updated size of the tokenizer's vocabulary
print(f"Updated vocabulary size: {len(tokenizer)}")

if '[ENTITY1]' and '[ENTITY2]' in tokenizer.get_vocab():
    print("[ENTITY1] and [ENTITY2] are in the tokenizer's vocabulary.")
else:
   print("[ENTITY1] and [ENTITY2] are NOT in the tokenizer's vocabulary.")
"""

# Load the checkpoint
checkpoint_path = '/content/drive/MyDrive/Checkpoints/final_chekpoint.pth'
try:
    checkpoint = torch.load(checkpoint_path, map_location=device)

    # Load the state dictionary (adjust the model to fit exactly the checkpoint's structure)
    missing_keys, unexpected_keys = model.load_state_dict(checkpoint['model_state_dict'], strict=False)
    print(f"Missing keys: {missing_keys}")
    print(f"Unexpected keys: {unexpected_keys}")

    # Move the model to the designated device
    model.to(device)
    print("Model loaded and moved to device:", device)
except FileNotFoundError:
    print(f"Checkpoint file not found at {checkpoint_path}")
except Exception as e:
    print(f"An error occurred while loading the checkpoint: {e}")


Token '[ENTITY1]' has ID: 30
Token '[ENTITY2]' has ID: 31


Some weights of the model checkpoint at Rostlab/prot_bert were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Token embeddings resized to accommodate new tokens.
Updated vocabulary size: 32
[ENTITY1] and [ENTITY2] are in the tokenizer's vocabulary.
Checkpoint file not found at /content/drive/MyDrive/Checkpoints/final_chekpoint.pth


In [ ]:
from torch.utils.data import Dataset
import torch
from torch.nn.utils.rnn import pad_sequence

class ProteinInteractionDataset(Dataset):
    """
    A PyTorch Dataset class that handles protein interaction data.
    This dataset supports multiple modes to prepare data, including masked and local sequence configurations,
    facilitating different training approaches.
    """
    def __init__(self, dataframe, tokenizer, max_length=512, mask_probability=0.15, modes=None):
        """
        Initializes the ProteinInteractionDataset.
        Args:
            dataframe (pandas.DataFrame): The dataframe containing protein sequences.
            tokenizer (transformers.BertTokenizer): The tokenizer for encoding sequences.
            max_length (int): Maximum sequence length to which sequences will be padded/truncated.
            mask_probability (float): Probability of masking a token for training masked language models.
            modes (list of str): Specifies the modes of data preparation (e.g., 'global_masked', 'local').
        """
        assert 'Sequence_A' in dataframe.columns and 'Sequence_B' in dataframe.columns, "DataFrame must include 'Sequence_A' and 'Sequence_B' columns."
        self.dataframe = dataframe
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.mask_probability = mask_probability
        self.modes = modes if modes else ['global_masked']

    def __len__(self):
        """
        Returns the number of items in the dataset.
        """
        return len(self.dataframe)

    def __getitem__(self, idx):
        """
        Retrieves an item by index.
        Args:
            idx (int): Index of the item.

        Returns:
            A dictionary of processed features for each mode.
        """
        row = self.dataframe.iloc[idx]
        data = {}

        for mode in self.modes:
            if mode == 'global_masked':
                sequence = f"[CLS] {row['Sequence_A']} [ENTITY1] [SEP] {row['Sequence_B'] } [ENTITY2] [SEP] "
                input_ids, attention_mask, labels = self.random_mask_sequence(sequence)
                key_suffix = 'global_masked'
            elif mode == 'local':
                sequence = f"[CLS] {row['masked_sequence_A']} [ENTITY1] [SEP] {row['masked_sequence_B'] } [ENTITY2] [SEP]"
                input_ids, attention_mask = self.tokenize_sequence(sequence)
                labels = torch.full_like(input_ids, -100)
                key_suffix = 'local'

            data[f'input_ids_{key_suffix}'] = input_ids
            data[f'attention_mask_{key_suffix}'] = attention_mask
            data[f'labels_{key_suffix}'] = labels

        return data

    def tokenize_sequence(self, sequence):
        """
        Tokenizes a sequence using the provided tokenizer.
        Args:
            sequence (str): The sequence to tokenize.

        Returns:
            Tuple of tensors (input_ids, attention_mask).
        """
        encoded = self.tokenizer.encode_plus(
            sequence,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return encoded['input_ids'].squeeze(0), encoded['attention_mask'].squeeze(0)

    def random_mask_sequence(self, sequence):
        """
        Randomly masks tokens in a sequence based on the specified mask probability.
        Args:
            sequence (str): The sequence to mask.

        Returns:
            Tuple of tensors (input_ids, attention_mask, labels).
        """
        encoded = self.tokenizer.encode_plus(
            sequence,
            add_special_tokens=True,
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        input_ids = encoded['input_ids'].squeeze(0)
        labels = torch.full_like(input_ids, -100)

        # Decide where to mask tokens
        mask_indices = (torch.rand(input_ids.shape) < self.mask_probability) & (input_ids != self.tokenizer.pad_token_id)
        labels[mask_indices] = input_ids[mask_indices]

        # 80% of the time replace with [MASK]
        actual_mask = mask_indices & (torch.rand(input_ids.shape) < 0.8)
        input_ids[actual_mask] = self.tokenizer.mask_token_id

        # 20% of the time replace with a random token
        random_tokens = torch.randint(2, self.tokenizer.vocab_size, input_ids.shape)
        input_ids[mask_indices & ~actual_mask] = random_tokens[mask_indices & ~actual_mask]

        return input_ids, torch.ones_like(input_ids), labels



In [ ]:
from torch.nn.utils.rnn import pad_sequence

def collate_fn(batch):
    global_input_ids = [item['global_input_ids'] for item in batch]
    global_attention_masks = [item['global_attention_mask'] for item in batch]
    local_input_ids = [item['local_input_ids'] for item in batch]
    local_attention_masks = [item['local_attention_mask'] for item in batch]

    # Pad sequences within each batch to the longest sequence in that batch
    global_input_ids_padded = pad_sequence(global_input_ids, batch_first=True, padding_value=0)
    global_attention_masks_padded = pad_sequence(global_attention_masks, batch_first=True, padding_value=0)
    local_input_ids_padded = pad_sequence(local_input_ids, batch_first=True, padding_value=0)
    local_attention_masks_padded = pad_sequence(local_attention_masks, batch_first=True, padding_value=0)

    return {
        'global_input_ids': global_input_ids_padded,
        'global_attention_mask': global_attention_masks_padded,
        'local_input_ids': local_input_ids_padded,
        'local_attention_mask': local_attention_masks_padded
    }


#Training and Validation data creator:

In [ ]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

# Splitting the dataset into 80% training and 20% testing
train_df, test_df = train_test_split(pairs_df, test_size=0.2, random_state=42)

# Assuming ProteinInteractionDataset and tokenizer are correctly implemented and initialized
train_dataset = ProteinInteractionDataset(train_df, tokenizer)
test_dataset = ProteinInteractionDataset(test_df, tokenizer)

# DataLoader setup, possibly with a custom collation function if your dataset needs it
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn if 'collate_fn' in globals() else None)
test_loader = DataLoader(test_dataset, batch_size=16, collate_fn=collate_fn if 'collate_fn' in globals() else None)


#pretraining the bert model


#main class for training:
  1. Process two sets of sequences (global and local) using BERT to extract contextual embeddings.
  2. Integrate these two sets of embeddings using a custom attention mechanism that focuses on relevant parts of the global features for each part of the local features.
  3. Predict an output (like interaction sites or effects) using the combined features.

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch
from torch import nn

class SequenceProcessor(nn.Module):
    def __init__(self, model):
        """
        Initialize the sequence processor with a pre-trained model.
        :param model: A pre-trained model, typically a fine-tuned ProtBERT.
        """
        super(SequenceProcessor, self).__init__()
        self.model = model

    def forward(self, input_ids, attention_mask=None):
        """
        Forward pass to process sequences through the model.
        Extracts last hidden state as the feature representation of the input.
        :param input_ids: Input tokens IDs to the model.
        :param attention_mask: Mask to avoid processing padding as part of the input.
        """
        outputs = self.model(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state

class CustomAttention(nn.Module):
    def __init__(self, hidden_size):
        """
        Custom attention mechanism that models relationships between features.
        :param hidden_size: Size of the hidden layers in the ProtBERT model.
        """
        super(CustomAttention, self).__init__()
        self.key_layer = nn.Linear(hidden_size, hidden_size)
        self.query_layer = nn.Linear(hidden_size, hidden_size)
        self.value_layer = nn.Linear(hidden_size, hidden_size)
        self.softmax = nn.Softmax(dim=-1)
        self.context_layer = nn.Linear(hidden_size, hidden_size)

    def forward(self, global_features, local_features):
        """
        Forward pass through the attention mechanism.
        Calculates attention scores and returns a context vector.
        :param global_features: Features from the global sequence processor.
        :param local_features: Features from the local sequence processor.
        """
        keys = self.key_layer(global_features)
        queries = self.query_layer(local_features)
        values = self.value_layer(global_features)
        attention_scores = torch.matmul(queries, keys.transpose(-2, -1)) / (keys.size(-1) ** 0.5)
        attention_weights = self.softmax(attention_scores)
        context = torch.matmul(attention_weights, values)
        processed_context = self.context_layer(context)
        return processed_context

class ProteinInteractionModel(nn.Module):
    def __init__(self, model_path):
        """
        Main model to predict protein interactions using ProtBERT with custom attention.
        :param model_path: Path to the fine-tuned ProtBERT model.
        """
        super(ProteinInteractionModel, self).__init__()
        self.tokenizer = AutoTokenizer.from_pretrained(model_path)
        self.model = AutoModel.from_pretrained(model_path)
        hidden_size = self.model.config.hidden_size
        self.global_processor = SequenceProcessor(self.model)
        self.local_processor = SequenceProcessor(self.model)
        self.attention = CustomAttention(hidden_size)
        self.output_layer = nn.Linear(hidden_size, self.tokenizer.vocab_size)

    def forward(self, input_ids_global, attention_mask_global, input_ids_local=None, attention_mask_local=None):
        """
        Forward pass for the entire model. Processes both global and local features,
        applies attention between them, and produces prediction scores.
        :param input_ids_global: Input IDs for the global sequence.
        :param attention_mask_global: Attention mask for the global sequence.
        :param input_ids_local: Input IDs for the local sequence (used in training).
        :param attention_mask_local: Attention mask for the local sequence (used in training).
        """
        global_features = self.global_processor(input_ids_global, attention_mask_global)
        if self.training and input_ids_local is not None:
            local_features = self.local_processor(input_ids_local, attention_mask_local)
        else:
            local_features = global_features  # Use global features for local prediction during inference
        context_features = self.attention(global_features, local_features)
        prediction_scores = self.output_layer(context_features)
        return prediction_scores


# Model Setup

#Training Loop

In [ ]:
def __getitem__(self, idx):
    # Extract the row from the dataframe
    row = self.dataframe.iloc[idx]

    # Prepare global and local input sequences
    global_input = f"[CLS] {row['Sequence_A']} [ENTITY1] [SEP] {row['Sequence_B']} [ENTITY2] [SEP]"
    local_input = f"[CLS] {row['masked_sequence_A']} [ENTITY1] [SEP] {row['masked_sequence_B']} [ENTITY2] [SEP]"

    # Tokenize sequences
    global_features = self.tokenize_sequence(global_input)
    local_features = self.tokenize_sequence(local_input)

    # Debugging: Print shapes and sample contents
    print("Debugging __getitem__:")
    print(f"Index: {idx}")
    print(f"Global Input IDs Shape: {global_features['input_ids'].shape}, Sample: {global_features['input_ids'][:10]}")
    print(f"Local Input IDs Shape: {local_features['input_ids'].shape}, Sample: {local_features['input_ids'][:10]}")

    # Return the processed features
    return {
        'global_input_ids': global_features['input_ids'],
        'global_attention_mask': global_features['attention_mask'],
        'local_input_ids': local_features['input_ids'],
        'local_attention_mask': local_features['attention_mask']
    }


In [ ]:
import torch
import torch.nn as nn
from torch.optim import Adam
from torch.utils.data import DataLoader

# Assume `ProteinInteractionModel` and `ProteinInteractionDataset` are defined in your code.
# Assume `collate_fn` is your custom collate function for handling batches.

# Setup Dataset and DataLoader
dataset = ProteinInteractionDataset(dataframe=pairs_df, tokenizer=tokenizer)  # Using the pre-loaded tokenizer
data_loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=collate_fn)

# Model and tokenizer are already initialized and loaded from the previous script segment
# Ensure the model is on the correct device
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = Adam(model.parameters(), lr=0.001)

# Move model to the appropriate device
model.to(device)

# Training loop
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    total_loss = 0

    for batch in data_loader:
        global_input_ids = batch['global_input_ids'].to(device)
        local_input_ids = batch['local_input_ids'].to(device)
        attention_mask_global = batch['global_attention_mask'].to(device)

        # Forward pass
        outputs = model(global_input_ids, attention_mask=attention_mask_global)
        logits = outputs.logits

        # Ensure the target labels are the same shape as the logits output
        # Reshape local_input_ids to be [batch_size * sequence_length]
        # This requires local_input_ids initially to be [batch_size, sequence_length]
        if local_input_ids.dim() == 2:
            local_input_ids = local_input_ids.view(-1)  # Flatten the local_input_ids

        # Calculate loss
        loss = criterion(logits.view(-1, logits.size(-1)), local_input_ids)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    print(f'Epoch {epoch+1}, Average Loss: {total_loss / len(data_loader)}')

# Optionally save the model after training
torch.save(model.state_dict(), '/content/drive/MyDrive/SecondModel.pth')


NameError: name 'row' is not defined